# CNN

Here we make a model 

In [1]:
# Check number of available GPUs
import tensorflow as tf
from tensorflow import keras
import os
from sklearn.preprocessing import StandardScaler
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
seed = 9+10
pixel_size=28
tf.random.set_seed(seed)

2025-11-08 21:14:01.609861: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-08 21:14:01.655029: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-08 21:14:07.597594: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Num GPUs Available:  1


In [2]:
def create_model(num_classes=10, optimizer=keras.optimizers.Adam(learning_rate=0.0001), shape = (pixel_size, pixel_size, 1)):
    lr_scheduler = keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.1,          # halve the learning rate if there is no improvement
        patience=5,          # Wait 5 epochs with no improvement before reducing
        min_lr=1e-6          # Set a minimum learning rate at 1e-6
    )
    early_stopper = keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=7,          # Wait 7 epochs for improvement before stopping
        restore_best_weights=True  # Automatically restore the weights from the best epoch
    )
    csv_logger = keras.callbacks.CSVLogger(
    filename=f"classification_log{num_classes}_classes.csv",
    separator=",",
    append=True)  #makes sure results are appended to same file if training stops and is resumed
    initializer = "glorot_uniform"# keras.initializers.Orthogonal(gain = 1.0, seed = seed)
    model= keras.models.Sequential([
        keras.Input(shape=shape),
        # Block 1
        keras.layers.Conv2D(32, (3,3), activation="relu", padding="same", kernel_initializer=initializer),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(32, (3,3), activation="relu", padding="same", kernel_initializer=initializer),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D((2,2)),
        keras.layers.Dropout(0.1),
        # Block 2
        keras.layers.Conv2D(64, (3,3), activation="relu", padding="same", kernel_initializer=initializer),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(64, (3,3), activation="relu", padding="same", kernel_initializer=initializer),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D(2),
        keras.layers.Dropout(0.1),
        # Block 3
        keras.layers.Conv2D(128, (3,3), activation="relu", padding="same", kernel_initializer=initializer),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(128, (3,3), activation="relu", padding="same", kernel_initializer=initializer),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D(2),
        keras.layers.Dropout(0.1),
        # Block 4
        keras.layers.Conv2D(256, (3,3), activation="relu", padding="same", kernel_initializer=initializer),
        keras.layers.BatchNormalization(),
        keras.layers.Conv2D(256, (3,3), activation="relu", padding="same", kernel_initializer=initializer),
        keras.layers.BatchNormalization(),
        keras.layers.MaxPooling2D(2),
        keras.layers.Dropout(0.1),


        keras.layers.Flatten(),
        keras.layers.Dense(128, activation="leaky_relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(64, activation="leaky_relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(int(num_classes), activation="softmax")
    ])
    model.compile(loss='sparse_categorical_crossentropy',
    optimizer=optimizer,
    metrics=["Accuracy"
            ],
    )
    return model, lr_scheduler, early_stopper, csv_logger


I0000 00:00:1762632851.422311 3832246 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6071 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060, pci bus id: 0000:01:00.0, compute capability: 8.9


## Model 1

In [3]:
# Import FashionMNIST data
fashion_mnist= keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
print(X_train_full.shape, y_train_full.shape)

# Preprocess data
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

X_test = X_test / 255.0
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)

pixel_size = 28
model1, lr_scheduler, early_stopper, csv_logger  = create_model(optimizer=keras.optimizers.Adam(learning_rate=0.0001))

(60000, 28, 28) (60000,)
(55000, 28, 28) (55000,)
(5000, 28, 28) (5000,)


In [4]:
model1.summary()
model1.fit(
    X_train, y_train,
    epochs=30,
    validation_data=(X_valid, y_valid),
    callbacks=[lr_scheduler, early_stopper, csv_logger]
)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 28, 28, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 28, 28, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 28, 28, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 14, 14, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 14, 14, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 7, 7, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 7, 7, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 7, 7, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 7, 7, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 3, 3, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 3, 3, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 3, 3, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 1,217,322 (4.64 MB)

 Trainable params: 1,215,402 (4.64 MB)

 Non-trainable params: 1,920 (7.50 KB)

...

  24/1719 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - Accuracy: 0.1550 - loss: 3.2391  

I0000 00:00:1762632865.480657 3832379 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1716/1719 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - Accuracy: 0.6250 - loss: 1.1236

2025-11-08 21:14:37.735844: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-11-08 21:14:37.735875: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-11-08 21:14:37.735927: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-11-08 21:14:37.996536: I external/l

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 36s 14ms/step - Accuracy: 0.7329 - loss: 0.7717 - val_Accuracy: 0.8678 - val_loss: 0.3750 - learning_rate: 1.0000e-04
Epoch 2/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - Accuracy: 0.8460 - loss: 0.4404 - val_Accuracy: 0.8880 - val_loss: 0.3042 - learning_rate: 1.0000e-04
Epoch 3/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - Accuracy: 0.8732 - loss: 0.3574 - val_Accuracy: 0.8980 - val_loss: 0.2719 - learning_rate: 1.0000e-04
Epoch 4/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - Accuracy: 0.8915 - loss: 0.3062 - val_Accuracy: 0.9032 - val_loss: 0.2720 - learning_rate: 1.0000e-04
Epoch 5/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - Accuracy: 0.9017 - loss: 0.2754 - val_Accuracy: 0.9152 - val_loss: 0.2325 - learning_rate: 1.0000e-04
Epoch 6/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - Accuracy: 0.9131 - loss: 0.2448 - val_Accuracy: 0.9202 - val_loss: 0.2195 - learning_rate: 1.0000e-04
Epoch 7/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step -

In [5]:
# Evaluate the model on the test set
test_loss, test_acc = model1.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - Accuracy: 0.9224 - loss: 0.2287
Test accuracy: 0.9223999977111816
Test loss: 0.2286963313817978


### Application to CIFAR10

In [6]:
# Import CIFAR10 dataset
CIFAR10 = tf.keras.datasets.cifar10


# Preprocess data
(X_train_full, y_train_full), (X_test, y_test) = CIFAR10.load_data()
print(X_train_full.shape, y_train_full.shape)

X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

X_test = X_test / 255.0
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)

(50000, 32, 32, 3) (50000, 1)
(45000, 32, 32, 3) (45000, 1)
(5000, 32, 32, 3) (5000, 1)


In [7]:
pixel_size = 32
modelA, lr_scheduler, early_stopper, csv_logger  = create_model(optimizer=keras.optimizers.Adam(learning_rate=0.0001),shape=(pixel_size, pixel_size, 3))

modelA.summary()
modelA.fit(
    X_train, y_train,
    epochs=30,
    validation_data=(X_valid, y_valid),
    callbacks=[lr_scheduler, early_stopper, csv_logger]
)

# Evaluate the model on the test set
test_loss, test_acc = modelA.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_8           │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 16, 16, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 8, 8, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 4, 4, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 4, 4, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 4, 4, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 1,316,202 (5.02 MB)

 Trainable params: 1,314,282 (5.01 MB)

 Non-trainable params: 1,920 (7.50 KB)

Epoch 1/30


2025-11-08 21:17:59.269092: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2025-11-08 21:17:59.858266: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_5588', 520 bytes spill stores, 520 bytes spill loads



1407/1407 ━━━━━━━━━━━━━━━━━━━━ 33s 15ms/step - Accuracy: 0.3456 - loss: 1.8227 - val_Accuracy: 0.5124 - val_loss: 1.3629 - learning_rate: 1.0000e-04
Epoch 2/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - Accuracy: 0.4987 - loss: 1.4043 - val_Accuracy: 0.5798 - val_loss: 1.1648 - learning_rate: 1.0000e-04
Epoch 3/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - Accuracy: 0.5786 - loss: 1.1957 - val_Accuracy: 0.6454 - val_loss: 0.9981 - learning_rate: 1.0000e-04
Epoch 4/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - Accuracy: 0.6285 - loss: 1.0586 - val_Accuracy: 0.6784 - val_loss: 0.9015 - learning_rate: 1.0000e-04
Epoch 5/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - Accuracy: 0.6733 - loss: 0.9342 - val_Accuracy: 0.6928 - val_loss: 0.8578 - learning_rate: 1.0000e-04
Epoch 6/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - Accuracy: 0.7079 - loss: 0.8395 - val_Accuracy: 0.7454 - val_loss: 0.7272 - learning_rate: 1.0000e-04
Epoch 7/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - 

## Model 2

In [8]:
# Import FashionMNIST data
fashion_mnist= keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
print(X_train_full.shape, y_train_full.shape)

# Preprocess data
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

X_test = X_test / 255.0
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)

pixel_size = 28
model2, lr_scheduler, early_stopper, csv_logger  = create_model(optimizer=keras.optimizers.AdamW(learning_rate=0.0001))

(60000, 28, 28) (60000,)
(55000, 28, 28) (55000,)
(5000, 28, 28) (5000,)


In [9]:
model2.summary()
model2.fit(
    X_train, y_train,
    epochs=30,
    validation_data=(X_valid, y_valid),
    callbacks=[lr_scheduler, early_stopper, csv_logger]
)

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_16 (Conv2D)              │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 28, 28, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 28, 28, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 28, 28, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 14, 14, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 14, 14, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_19          │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 7, 7, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_20          │ (None, 7, 7, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 7, 7, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_21          │ (None, 7, 7, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_22 (Conv2D)              │ (None, 3, 3, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_22          │ (None, 3, 3, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_23 (Conv2D)              │ (None, 3, 3, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 1,217,322 (4.64 MB)

 Trainable params: 1,215,402 (4.64 MB)

 Non-trainable params: 1,920 (7.50 KB)

Epoch 1/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 32s 13ms/step - Accuracy: 0.7368 - loss: 0.7630 - val_Accuracy: 0.8628 - val_loss: 0.3798 - learning_rate: 1.0000e-04
Epoch 2/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - Accuracy: 0.8486 - loss: 0.4378 - val_Accuracy: 0.8892 - val_loss: 0.3011 - learning_rate: 1.0000e-04
Epoch 3/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - Accuracy: 0.8773 - loss: 0.3557 - val_Accuracy: 0.9026 - val_loss: 0.2674 - learning_rate: 1.0000e-04
Epoch 4/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 11s 6ms/step - Accuracy: 0.8929 - loss: 0.3062 - val_Accuracy: 0.9104 - val_loss: 0.2410 - learning_rate: 1.0000e-04
Epoch 5/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - Accuracy: 0.9045 - loss: 0.2738 - val_Accuracy: 0.9094 - val_loss: 0.2428 - learning_rate: 1.0000e-04
Epoch 6/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 15s 9ms/step - Accuracy: 0.9135 - loss: 0.2457 - val_Accuracy: 0.9166 - val_loss: 0.2350 - learning_rate: 1.0000e-04
Epoch 7/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 12s

In [10]:
# Evaluate the model on the test set
test_loss, test_acc = model2.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - Accuracy: 0.9270 - loss: 0.2331
Test accuracy: 0.9269999861717224
Test loss: 0.23312963545322418


### Application to CIFAR10

In [11]:
# Import CIFAR10 dataset
CIFAR10 = tf.keras.datasets.cifar10


# Preprocess data
(X_train_full, y_train_full), (X_test, y_test) = CIFAR10.load_data()
print(X_train_full.shape, y_train_full.shape)

X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

X_test = X_test / 255.0
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)

(50000, 32, 32, 3) (50000, 1)
(45000, 32, 32, 3) (45000, 1)
(5000, 32, 32, 3) (5000, 1)


In [12]:
pixel_size = 32
modelB, lr_scheduler, early_stopper, csv_logger  = create_model(optimizer=keras.optimizers.AdamW(learning_rate=0.0001),shape=(pixel_size, pixel_size, 3))

modelB.summary()
modelB.fit(
    X_train, y_train,
    epochs=30,
    validation_data=(X_valid, y_valid),
    callbacks=[lr_scheduler, early_stopper, csv_logger]
)

# Evaluate the model on the test set
test_loss, test_acc = modelB.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_24 (Conv2D)              │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_24          │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_25 (Conv2D)              │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_25          │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_26          │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_27 (Conv2D)              │ (None, 16, 16, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_27          │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_28 (Conv2D)              │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_28          │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 8, 8, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_29          │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_30 (Conv2D)              │ (None, 4, 4, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_30          │ (None, 4, 4, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_31 (Conv2D)              │ (None, 4, 4, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 1,316,202 (5.02 MB)

 Trainable params: 1,314,282 (5.01 MB)

 Non-trainable params: 1,920 (7.50 KB)

Epoch 1/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 31s 14ms/step - Accuracy: 0.3463 - loss: 1.8228 - val_Accuracy: 0.5002 - val_loss: 1.4149 - learning_rate: 1.0000e-04
Epoch 2/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - Accuracy: 0.4960 - loss: 1.4086 - val_Accuracy: 0.6026 - val_loss: 1.1199 - learning_rate: 1.0000e-04
Epoch 3/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - Accuracy: 0.5773 - loss: 1.1971 - val_Accuracy: 0.6444 - val_loss: 0.9987 - learning_rate: 1.0000e-04
Epoch 4/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - Accuracy: 0.6282 - loss: 1.0585 - val_Accuracy: 0.6862 - val_loss: 0.8824 - learning_rate: 1.0000e-04
Epoch 5/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - Accuracy: 0.6770 - loss: 0.9333 - val_Accuracy: 0.7260 - val_loss: 0.8063 - learning_rate: 1.0000e-04
Epoch 6/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - Accuracy: 0.7110 - loss: 0.8354 - val_Accuracy: 0.7314 - val_loss: 0.7605 - learning_rate: 1.0000e-04
Epoch 7/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 10s

## Model 3

In [13]:
# Import FashionMNIST data
fashion_mnist= keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
print(X_train_full.shape, y_train_full.shape)

# Preprocess data
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

X_test = X_test / 255.0
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)

pixel_size = 28
model3, lr_scheduler, early_stopper, csv_logger  = create_model(optimizer=keras.optimizers.Nadam(learning_rate=0.0001))

(60000, 28, 28) (60000,)
(55000, 28, 28) (55000,)
(5000, 28, 28) (5000,)


In [14]:
model3.summary()
model3.fit(
    X_train, y_train,
    epochs=30,
    validation_data=(X_valid, y_valid),
    callbacks=[lr_scheduler, early_stopper, csv_logger]
)

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_32 (Conv2D)              │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_32          │ (None, 28, 28, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_33 (Conv2D)              │ (None, 28, 28, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_33          │ (None, 28, 28, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_24 (Dropout)            │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_34 (Conv2D)              │ (None, 14, 14, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_34          │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_35 (Conv2D)              │ (None, 14, 14, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_35          │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_25 (Dropout)            │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_36 (Conv2D)              │ (None, 7, 7, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_36          │ (None, 7, 7, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_37 (Conv2D)              │ (None, 7, 7, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_37          │ (None, 7, 7, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_38 (Conv2D)              │ (None, 3, 3, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_38          │ (None, 3, 3, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_39 (Conv2D)              │ (None, 3, 3, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 1,217,322 (4.64 MB)

 Trainable params: 1,215,402 (4.64 MB)

 Non-trainable params: 1,920 (7.50 KB)

Epoch 1/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 34s 13ms/step - Accuracy: 0.7412 - loss: 0.7572 - val_Accuracy: 0.8704 - val_loss: 0.3724 - learning_rate: 1.0000e-04
Epoch 2/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - Accuracy: 0.8510 - loss: 0.4340 - val_Accuracy: 0.8962 - val_loss: 0.2921 - learning_rate: 1.0000e-04
Epoch 3/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - Accuracy: 0.8768 - loss: 0.3556 - val_Accuracy: 0.9032 - val_loss: 0.2705 - learning_rate: 1.0000e-04
Epoch 4/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - Accuracy: 0.8927 - loss: 0.3069 - val_Accuracy: 0.9132 - val_loss: 0.2392 - learning_rate: 1.0000e-04
Epoch 5/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - Accuracy: 0.9033 - loss: 0.2741 - val_Accuracy: 0.9154 - val_loss: 0.2312 - learning_rate: 1.0000e-04
Epoch 6/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - Accuracy: 0.9103 - loss: 0.2476 - val_Accuracy: 0.9192 - val_loss: 0.2223 - learning_rate: 1.0000e-04
Epoch 7/30
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 12s

In [15]:
# Evaluate the model on the test set
test_loss, test_acc = model3.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - Accuracy: 0.9221 - loss: 0.2373
Test accuracy: 0.9221000075340271
Test loss: 0.2372908890247345


#### Application to CIFAR10

In [16]:
# Import CIFAR10 dataset
CIFAR10 = tf.keras.datasets.cifar10


# Preprocess data
(X_train_full, y_train_full), (X_test, y_test) = CIFAR10.load_data()
print(X_train_full.shape, y_train_full.shape)

X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

X_test = X_test / 255.0
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)

(50000, 32, 32, 3) (50000, 1)
(45000, 32, 32, 3) (45000, 1)
(5000, 32, 32, 3) (5000, 1)


In [17]:
pixel_size = 32
modelC, lr_scheduler, early_stopper, csv_logger  = create_model(optimizer=keras.optimizers.AdamW(learning_rate=0.0001),shape=(pixel_size, pixel_size, 3))

modelC.summary()
modelC.fit(
    X_train, y_train,
    epochs=30,
    validation_data=(X_valid, y_valid),
    callbacks=[lr_scheduler, early_stopper, csv_logger]
)

# Evaluate the model on the test set
test_loss, test_acc = modelC.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_40 (Conv2D)              │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_40          │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_41 (Conv2D)              │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_41          │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_30 (Dropout)            │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_42 (Conv2D)              │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_42          │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_43 (Conv2D)              │ (None, 16, 16, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_43          │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_21 (MaxPooling2D) │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_31 (Dropout)            │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_44 (Conv2D)              │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_44          │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_45 (Conv2D)              │ (None, 8, 8, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_45          │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_22 (MaxPooling2D) │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_32 (Dropout)            │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_46 (Conv2D)              │ (None, 4, 4, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_46          │ (None, 4, 4, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_47 (Conv2D)              │ (None, 4, 4, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 1,316,202 (5.02 MB)

 Trainable params: 1,314,282 (5.01 MB)

 Non-trainable params: 1,920 (7.50 KB)

Epoch 1/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 31s 14ms/step - Accuracy: 0.3535 - loss: 1.8262 - val_Accuracy: 0.5042 - val_loss: 1.3889 - learning_rate: 1.0000e-04
Epoch 2/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - Accuracy: 0.4943 - loss: 1.4230 - val_Accuracy: 0.6004 - val_loss: 1.1272 - learning_rate: 1.0000e-04
Epoch 3/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - Accuracy: 0.5699 - loss: 1.2180 - val_Accuracy: 0.6404 - val_loss: 1.0046 - learning_rate: 1.0000e-04
Epoch 4/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - Accuracy: 0.6253 - loss: 1.0643 - val_Accuracy: 0.6886 - val_loss: 0.8814 - learning_rate: 1.0000e-04
Epoch 5/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 9s 6ms/step - Accuracy: 0.6691 - loss: 0.9458 - val_Accuracy: 0.7136 - val_loss: 0.8133 - learning_rate: 1.0000e-04
Epoch 6/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - Accuracy: 0.7056 - loss: 0.8447 - val_Accuracy: 0.7230 - val_loss: 0.7897 - learning_rate: 1.0000e-04
Epoch 7/30
1407/1407 ━━━━━━━━━━━━━━━━━━━━ 9s 6m